In [1]:
# Setup
import sys
from pathlib import Path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from core.multi_asset_loader import load_assets
from core.risk_manager import RiskManager, RiskConfig

print("✅ Imports successful")

✅ Imports successful


---

## Step 1: Create RiskManager

**Source:** `core/risk_manager.py` lines 11-27

**Key Parameters:**
- `max_position_size` - Max % of portfolio per asset (e.g., 0.35 = 35%)
- `max_drawdown` - Stop trading if drawdown exceeds (e.g., 0.20 = 20%)
- `position_sizing_method` - 'equal', 'vol_adjusted', or 'kelly'
- `target_volatility` - For vol_adjusted (e.g., 0.15 = 15% annual)
- `max_correlation` - Reject new positions if avg correlation too high

In [4]:
# Create risk configuration
risk_config = RiskConfig(
    max_position_size=0.35,        # 35% max per asset
    max_drawdown_stop=0.20,             # 20% drawdown stop
    position_sizing_method='vol_adjusted',
    volatility_target=0.15,        # 15% annual vol target
    correlation_threshold=0.70,          # 70% correlation limit
    correlation_window=60             # 60 days for vol/correlation calc
)

# Create risk manager
rm = RiskManager(risk_config)

print("\n🛡️ RiskManager Created:")
print(f"  Max Position Size: {rm.config.max_position_size:.0%}")
print(f"  Max Drawdown: {rm.config.max_drawdown_stop:.0%}")
print(f"  Sizing Method: {rm.config.position_sizing_method}")
print(f"  Target Volatility: {rm.config.volatility_target:.0%}")
print(f"  Max Correlation: {rm.config.correlation_threshold:.0%}")
print(f"  Lookback Period: {rm.config.correlation_window} days")


🛡️ RiskManager Created:
  Max Position Size: 35%
  Max Drawdown: 20%
  Sizing Method: vol_adjusted
  Target Volatility: 15%
  Max Correlation: 70%
  Lookback Period: 60 days


---

## Step 2: Load Data for Risk Calculations

We need price history to calculate volatility and correlations.

In [5]:
# Load multi-asset data
TICKERS = ['ES', 'GC', 'NQ']
prices = load_assets(
    tickers=TICKERS,
    start_date='2024-01-01',
    end_date='2024-12-31'
)

print("\n📊 Loaded Data:")
for ticker, df in prices.items():
    print(f"  {ticker}: {len(df)} days")


Loading 3 assets: ['ES', 'GC', 'NQ']
  📡 CSV data is 330 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-26)
✓ ES: 6362 rows, 2000-09-18 to 2025-11-26
  📡 CSV data is 330 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-26)
✓ GC: 6334 rows, 2000-08-30 to 2025-11-26
  📡 CSV data is 330 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-26)
✓ ES: 6362 rows, 2000-09-18 to 2025-11-26
  📡 CSV data is 330 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-26)
✓ GC: 6334 rows, 2000-08-30 to 2025-11-26
  📡 CSV data is 330 days old, fetching recent data from yfinance...
  ✓ Added 228 days from yfinance (now through 2025-11-26)
✓ NQ: 6362 rows, 2000-09-18 to 2025-11-26

Common date range: 2000-09-18 to 2025-11-26
Filtered date range: 2024-01-01 to 2024-12-31

Total unique dates: 252

ALIGN

---

## Step 3: Calculate Volatility

**Source:** `core/risk_manager.py` → `calculate_volatility()`

**Formula:**
```
daily_returns = price.pct_change()
vol_daily = returns.std()
vol_annual = vol_daily × sqrt(252)
```

**252 = typical trading days per year**

In [7]:
# Calculate volatility for each asset
print("\n📈 Calculating Volatility (60-day lookback):")
print(f"\n{'Ticker':<8} {'Daily Vol':>12} {'Annual Vol':>12}")
print("-" * 35)

volatilities = {}
for ticker in TICKERS:
    df = prices[ticker]
    returns = df['Close'].pct_change().iloc[-60:]  # Last 60 days
    vol_daily = returns.std()
    vol_annual = vol_daily * np.sqrt(252)
    volatilities[ticker] = vol_annual
    
    print(f"{ticker:<8} {vol_daily:>11.2%} {vol_annual:>11.1%}")

print(f"\n💡 Target Volatility: {risk_config.volatility_target:.0%}")


📈 Calculating Volatility (60-day lookback):

Ticker      Daily Vol   Annual Vol
-----------------------------------
ES             0.90%       14.3%
GC             1.08%       17.1%
NQ             1.20%       19.1%

💡 Target Volatility: 15%


---

## Step 4: Position Sizing - Vol Adjusted

**Source:** `core/risk_manager.py` → `calculate_position_size()`

**Logic:**
```
target_vol = 15%
asset_vol = 25%

vol_scalar = target_vol / asset_vol = 0.60
base_size = 1 / n_assets = 0.33
adjusted_size = base_size × vol_scalar = 0.20

capped_size = min(adjusted_size, max_position_size) = 0.20
```

**Higher vol → Smaller position**

In [10]:
# Calculate position sizes
n_active = len(TICKERS)
base_size = 1 / n_active

print(f"\n⚖️ Vol-Adjusted Position Sizing:")
print(f"\nBase Size (equal weight): {base_size:.1%}")
print(f"Target Volatility: {risk_config.volatility_target:.0%}")
print(f"Max Position Size: {risk_config.max_position_size:.0%}")

print(f"\n{'Ticker':<8} {'Asset Vol':>12} {'Vol Scalar':>12} {'Adjusted':>12} {'Final':>12}")
print("-" * 60)

for ticker in TICKERS:
    asset_vol = volatilities[ticker]
    vol_scalar = risk_config.volatility_target / asset_vol if asset_vol > 0 else 1.0
    adjusted = base_size * vol_scalar
    final = min(adjusted, risk_config.max_position_size)
    
    print(f"{ticker:<8} {asset_vol:>11.1%} {vol_scalar:>11.2f} {adjusted:>11.1%} {final:>11.1%}")

print("\n💡 Lower volatility assets get larger positions!")


⚖️ Vol-Adjusted Position Sizing:

Base Size (equal weight): 33.3%
Target Volatility: 15%
Max Position Size: 35%

Ticker      Asset Vol   Vol Scalar     Adjusted        Final
------------------------------------------------------------
ES             14.3%        1.05       35.0%       35.0%
GC             17.1%        0.88       29.3%       29.3%
NQ             19.1%        0.79       26.2%       26.2%

💡 Lower volatility assets get larger positions!


---

## Step 5: Correlation Matrix

**Source:** `core/risk_manager.py` → `calculate_correlation_matrix()`

**Why?** Avoid over-concentration in correlated assets.

**Example:**
- ES and NQ (both stock indices) → High correlation
- ES and GC (stocks vs gold) → Low correlation

In [14]:
# Calculate correlation matrix
returns_df = pd.DataFrame({
    ticker: prices[ticker]['Close'].pct_change().iloc[-60:]
    for ticker in TICKERS
})

corr_matrix = returns_df.corr()

print("\n🔗 Correlation Matrix (60-day):")
print(corr_matrix.round(2))

# Average correlation for each asset with others
print("\n📊 Average Correlation with Other Assets:")
for ticker in TICKERS:
    others = [t for t in TICKERS if t != ticker]
    avg_corr = corr_matrix.loc[ticker, others].mean()
    status = '✅' if avg_corr < risk_config.correlation_threshold else '⚠️'
    print(f"  {status} {ticker}: {avg_corr:.2f}")

print(f"\n🎯 Max Allowed Correlation: {risk_config.correlation_threshold:.2f}")


🔗 Correlation Matrix (60-day):
      ES    GC    NQ
ES  1.00 -0.00  0.95
GC -0.00  1.00  0.05
NQ  0.95  0.05  1.00

📊 Average Correlation with Other Assets:
  ✅ ES: 0.47
  ✅ GC: 0.02
  ✅ NQ: 0.50

🎯 Max Allowed Correlation: 0.70


---

## Step 6: Drawdown Monitoring

**Source:** `core/risk_manager.py` → `update_drawdown()` and `check_drawdown_stop()`

**Formula:**
```
peak = max(equity_curve)
current = latest_equity
drawdown = (current - peak) / peak

if drawdown < -max_drawdown:
    STOP TRADING!
```

In [18]:
# Simulate equity curve
equity_curve = [
    100000,  # Start
    105000,  # Up 5%
    110000,  # Up 10% (peak)
    108000,  # Down 1.8%
    102000,  # Down 7.3%
    95000,   # Down 13.6%
    88000,   # Down 20% (STOP!)
]

print("\n📉 Drawdown Monitoring:")
print(f"\n{'Step':<6} {'Equity':>12} {'Peak':>12} {'Drawdown':>12} {'Status':>15}")
print("-" * 60)

peak = 0
for i, equity in enumerate(equity_curve):
    peak = max(peak, equity)
    drawdown = (equity - peak) / peak if peak > 0 else 0
    
    if drawdown < -risk_config.max_drawdown_stop:
        status = "🛑 STOP!"
    elif drawdown < -risk_config.max_drawdown_stop * 0.75:
        status = "⚠️ Warning"
    else:
        status = "✅ OK"
    
    print(f"{i+1:<6} ${equity:>11,} ${peak:>11,} {drawdown:>11.1%} {status:>15}")

print(f"\n🎯 Max Drawdown Limit: {risk_config.max_drawdown_stop:.0%}")
print("\n💡 When limit breached, RiskManager stops all new entries!")


📉 Drawdown Monitoring:

Step         Equity         Peak     Drawdown          Status
------------------------------------------------------------
1      $    100,000 $    100,000        0.0%            ✅ OK
2      $    105,000 $    105,000        0.0%            ✅ OK
3      $    110,000 $    110,000        0.0%            ✅ OK
4      $    108,000 $    110,000       -1.8%            ✅ OK
5      $    102,000 $    110,000       -7.3%            ✅ OK
6      $     95,000 $    110,000      -13.6%            ✅ OK
7      $     88,000 $    110,000      -20.0%      ⚠️ Warning

🎯 Max Drawdown Limit: 20%

💡 When limit breached, RiskManager stops all new entries!


---

## Step 7: Risk Violations

**Source:** `core/risk_manager.py` → `validate_trade()`

**Checks:**
1. Position size > max_position_size?
2. Drawdown > max_drawdown?
3. Avg correlation > max_correlation?

**If violated:** Block new positions, force rebalance, or exit all.

In [19]:
# Example: Check violations
current_positions = {
    'ES': {'weight': 0.40},  # Violates max_position_size!
    'GC': {'weight': 0.30},
    'NQ': {'weight': 0.30}
}

print("\n🚨 Violation Check:")
print(f"\nMax Position Size: {risk_config.max_position_size:.0%}")

violations = []
for ticker, pos in current_positions.items():
    if pos['weight'] > risk_config.max_position_size:
        violations.append(f"{ticker}: {pos['weight']:.0%} > {risk_config.max_position_size:.0%}")
        print(f"  ❌ {ticker}: {pos['weight']:.0%} exceeds limit!")
    else:
        print(f"  ✅ {ticker}: {pos['weight']:.0%} within limit")

if violations:
    print(f"\n⚠️ {len(violations)} violation(s) detected")
    print("   Action: Rebalance to bring ES down to 35%")
else:
    print("\n✅ No violations")


🚨 Violation Check:

Max Position Size: 35%
  ❌ ES: 40% exceeds limit!
  ✅ GC: 30% within limit
  ✅ NQ: 30% within limit

⚠️ 1 violation(s) detected
   Action: Rebalance to bring ES down to 35%


---

## Step 8: Integration with PortfolioManager

**How it works:**

```python
# In PortfolioManager
if self.risk_manager:
    # 1. Ask risk manager for position sizes
    sizes = risk_manager.calculate_position_size(
        signals, prices, equity_curve
    )
    
    # 2. Check violations before trading
    violations = risk_manager.check_violations(
        positions, equity_curve
    )
    
    if violations:
        # Don't trade! Or force exit.
        return
```

**RiskManager acts as a gatekeeper!**

In [20]:
# Show integration points
print("\n🔗 RiskManager Integration Points:")
print("\n1. Position Sizing:")
print("   PortfolioConfig(risk_manager=rm)")
print("   → Calls rm.calculate_position_size() during rebalance")

print("\n2. Pre-Trade Checks:")
print("   Before executing trades:")
print("   → Check rm.check_drawdown_stop()")
print("   → Check rm.check_violations()")

print("\n3. Post-Trade Updates:")
print("   After updating positions:")
print("   → Call rm.update_drawdown(current_equity)")
print("   → Call rm.calculate_correlation_matrix(prices)")

print("\n💡 See core/portfolio_manager.py lines 120-160 for implementation")


🔗 RiskManager Integration Points:

1. Position Sizing:
   PortfolioConfig(risk_manager=rm)
   → Calls rm.calculate_position_size() during rebalance

2. Pre-Trade Checks:
   Before executing trades:
   → Check rm.check_drawdown_stop()
   → Check rm.check_violations()

3. Post-Trade Updates:
   After updating positions:
   → Call rm.update_drawdown(current_equity)
   → Call rm.calculate_correlation_matrix(prices)

💡 See core/portfolio_manager.py lines 120-160 for implementation


---

## 🎓 Key Takeaways

### RiskManager Architecture:

```
┌────────────────────────────────────┐
│       RiskManager                  │
│                                    │
│  State:                            │
│    • peak_equity                  │
│    • current_drawdown             │
│    • correlation_matrix           │
│                                    │
│  Methods:                          │
│    • calculate_position_size()    │
│    • calculate_volatility()       │
│    • calculate_correlation()      │
│    • update_drawdown()            │
│    • check_violations()           │
│    • check_drawdown_stop()        │
└────────────────────────────────────┘
```

### Position Sizing Methods:

1. **Equal Weight:**
   ```
   size = 1 / n_assets
   ```

2. **Vol Adjusted:**
   ```
   scalar = target_vol / asset_vol
   size = (1 / n_assets) × scalar
   ```

3. **Kelly Criterion:**
   ```
   edge = win_rate × avg_win - loss_rate × avg_loss
   kelly = edge / avg_win
   size = kelly × (1 / n_assets)
   ```

### Risk Controls:

| Control | Purpose | Action |
|---------|---------|--------|
| Max Position Size | Limit concentration | Cap at 35% |
| Max Drawdown | Preserve capital | Stop at -20% |
| Max Correlation | Diversification | Reject if > 0.70 |
| Volatility Target | Consistent risk | Scale by vol |

### Critical Insight:
⚠️ **RiskManager is OPTIONAL but RECOMMENDED**

Without it:
- Equal weight positions
- No drawdown stop
- No correlation check

With it:
- Adaptive sizing
- Capital preservation
- Better diversification

---

## ✅ Next Step

**Open: `04_full_system_integration.ipynb`**

Put it all together: Data → Signals → Risk → Portfolio → Backtest!